In [180]:
%pip install pandas-market-calendars
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_market_calendars as mcal


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [181]:
#Feature Engineering

#1. Percentage change
BIT_USD = pd.read_excel('Data/BIT_USD.xlsx', index_col=0)
#BIT_USD['pct_change'] = BIT_USD['Open'].pct_change()

BIT_USD['BTC_Log_Change'] = np.log(BIT_USD['Open'] / BIT_USD['Open'].shift(1))


#2. Bid-Ask Spread
BIT_USD['BTC_Bid_ask_spread'] = BIT_USD['Ask'] - BIT_USD['Bid']

missing_values_count = BIT_USD['BTC_Log_Change'].isna().sum()
print(f"Number of missing values in 'ColumnName': {missing_values_count}")
BIT_USD


Number of missing values in 'ColumnName': 1


,Local Time,Bid,Ask,High,Low,Open,Refresh Rate,BidNet,BTC_Log_Change,BTC_Bid_ask_spread
Local Date,,,,,,,,,,
2023-11-08 16:00:00,2023-11-08 16:00:00,35624.00,35625.00,35634.25,35541.92,35617.65,725,13.00,NaN,1.00
2023-11-08 15:55:00,2023-11-08 15:55:00,35611.00,35612.00,35621.75,35541.89,35569.00,784,41.63,-0.001367,1.00
2023-11-08 15:50:00,2023-11-08 15:50:00,35569.37,35572.72,35614.52,35516.60,35581.00,754,-8.66,0.000337,3.35
2023-11-08 15:45:00,2023-11-08 15:45:00,35578.03,35581.49,35585.75,35493.69,35493.69,793,63.93,-0.002457,3.46
2023-11-08 15:40:00,2023-11-08 15:40:00,35514.10,35517.03,35605.19,35447.68,35591.00,970,-87.57,0.002738,2.93
...,...,...,...,...,...,...,...,...,...,...
2022-11-21 09:50:00,2022-11-21 09:50:00,16186.00,16190.00,16191.00,16152.00,16160.01,728,25.99,-0.001607,4.00
2022-11-21 09:45:00,2022-11-21 09:45:00,16160.01,16170.30,16175.00,16144.07,16147.00,694,13.01,-0.000805,10.29
2022-11-21 09:40:00,2022-11-21 09:40:00,16147.00,16150.00,16168.50,16134.44,16153.00,771,-2.59,0.000372,3.00


In [182]:
BIT_USD.index = pd.to_datetime(BIT_USD.index)
nyse_calendar = mcal.get_calendar('NYSE')


start_date = pd.Timestamp('2022-11-21 09:30:00')
end_date = pd.Timestamp('2023-11-08 16:00:00')

schedule = nyse_calendar.schedule(start_date=start_date, end_date=end_date)
BIT_USD_filtered = BIT_USD[BIT_USD.index.normalize().isin(schedule.index.date)]

BIT_USD_filtered = BIT_USD_filtered.between_time('09:30:00', '16:00:00')

BIT_USD_filtered
BIT_USD_filtered.to_csv('Filtered_Data/BIT_USD_filtered.csv')
BIT_USD_filtered.shape

(19197, 10)

In [183]:
####################Done BTC####################

In [184]:
AAPL = pd.read_excel('Data/AAPL.xlsx', index_col=0)
AAPL.index = pd.to_datetime(AAPL.index)
AAPL
#Feature Engineering
#1. Log Change
AAPL['AAPL_Log_Change'] = np.log(AAPL['Close'] / AAPL['Close'].shift(1))

#2. Bid-Ask Spread
AAPL['AAPL_High_Low_Spread'] = AAPL['High'] - AAPL['Low']
AAPL['AAPL_Volume'] = AAPL['Volume']

#3. Moving Average
AAPL['AAPL_1hr_Moving_Average'] = AAPL['Open'].rolling(window=20).mean()

#4. RSI
def calculate_rsi(data, window=20):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

AAPL['AAPL_RSI'] = calculate_rsi(AAPL['Close'])
AAPL['AAPL_Volume'] = AAPL['Volume']

AAPL = AAPL.between_time('09:30:00', '16:00:00')
missing_values_count = AAPL['AAPL_RSI'].isna().sum()
print(f"Number of missing values in 'ColumnName': {missing_values_count}")
AAPL
AAPL.to_csv('Filtered_Data/AAPL_filtered.csv')


AAPL.index = pd.to_datetime(AAPL.index)
nyse_calendar = mcal.get_calendar('NYSE')


start_date = pd.Timestamp('2022-11-21 09:30:00')
end_date = pd.Timestamp('2023-11-08 16:00:00')

schedule = nyse_calendar.schedule(start_date=start_date, end_date=end_date)
AAPL_filtered = AAPL[AAPL.index.normalize().isin(schedule.index.date)]

AAPL_filtered = AAPL_filtered.between_time('09:30:00', '16:00:00')

AAPL_filtered
AAPL_filtered.to_csv('Filtered_Data/AAPL_filtered.csv')
AAPL.shape

Number of missing values in 'ColumnName': 19


(19197, 13)

In [185]:
####################Done AAPL####################

In [186]:
AMZN = pd.read_excel('Data/AMZN.xlsx', index_col=0)
AMZN.index = pd.to_datetime(AMZN.index)

#Feature Engineering
#1. Log Change
AMZN['AMZN_Log_Change'] = np.log(AMZN['Close'] / AMZN['Close'].shift(1))

#2. Bid-Ask Spread
AMZN['AMZN_High_Low_Spread'] = AMZN['High'] - AMZN['Low']
AMZN['AMZN_Volume'] = AMZN['Volume']

#3. Moving Average
AMZN['AMZN_1hr_Moving_Average'] = AMZN['Open'].rolling(window=20).mean()

#4. RSI
def calculate_rsi(data, window=20):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

AMZN['AMZN_RSI'] = calculate_rsi(AMZN['Close'])
AMZN['AMZN_Volume'] = AMZN['Volume']

AMZN = AMZN.between_time('09:30:00', '16:00:00')
missing_values_count = AMZN['AMZN_RSI'].isna().sum()
print(f"Number of missing values in 'ColumnName': {missing_values_count}")


AMZN.index = pd.to_datetime(AMZN.index)
nyse_calendar = mcal.get_calendar('NYSE')


start_date = pd.Timestamp('2022-11-21 09:30:00')
end_date = pd.Timestamp('2023-11-08 16:00:00')

schedule = nyse_calendar.schedule(start_date=start_date, end_date=end_date)
AMZN = AMZN[AMZN.index.normalize().isin(schedule.index.date)]

AMZN = AMZN.between_time('09:30:00', '16:00:00')

AMZN.to_csv('Filtered_Data/AMZN_filtered.csv')
AMZN.shape

Number of missing values in 'ColumnName': 19


(19197, 13)

In [187]:
####################Done AMZN####################

In [188]:
GOOGL = pd.read_excel('Data/GOOGL.xlsx', index_col=0)
GOOGL.index = pd.to_datetime(GOOGL.index)

#Feature Engineering
#1. Log Change
GOOGL['GOOGL_Log_Change'] = np.log(GOOGL['Close'] / GOOGL['Close'].shift(1))

#2. Bid-Ask Spread
GOOGL['GOOGL_High_Low_Spread'] = GOOGL['High'] - GOOGL['Low']
GOOGL['GOOGL_Volume'] = GOOGL['Volume']

#3. Moving Average
GOOGL['GOOGL_1hr_Moving_Average'] = GOOGL['Open'].rolling(window=20).mean()

#4. RSI
def calculate_rsi(data, window=20):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

GOOGL['GOOGL_RSI'] = calculate_rsi(GOOGL['Close'])
GOOGL['GOOGL_Volume'] = GOOGL['Volume']
GOOGL = GOOGL.between_time('09:30:00', '16:00:00')
missing_values_count = GOOGL['GOOGL_RSI'].isna().sum()
print(f"Number of missing values in 'ColumnName': {missing_values_count}")

GOOGL.index = pd.to_datetime(GOOGL.index)
nyse_calendar = mcal.get_calendar('NYSE')

start_date = pd.Timestamp('2022-11-21 09:30:00')
end_date = pd.Timestamp('2023-11-08 16:00:00')

schedule = nyse_calendar.schedule(start_date=start_date, end_date=end_date)
GOOGL = GOOGL[GOOGL.index.normalize().isin(schedule.index.date)]

GOOGL = GOOGL.between_time('09:30:00', '16:00:00')
GOOGL.to_csv('Filtered_Data/GOOGL_filtered.csv')
GOOGL.shape

Number of missing values in 'ColumnName': 19


(19197, 13)

In [189]:
####################Done GOOGL####################

In [190]:
MSFT = pd.read_excel('Data/MSFT.xlsx', index_col=0)
MSFT.index = pd.to_datetime(MSFT.index)

#Feature Engineering
#1. Log Change
MSFT['MSFT_Log_Change'] = np.log(MSFT['Close'] / MSFT['Close'].shift(1))

#2. Bid-Ask Spread
MSFT['MSFT_High_Low_Spread'] = MSFT['High'] - MSFT['Low']
MSFT['MSFT_Volume'] = MSFT['Volume']

#3. Moving Average
MSFT['MSFT_1hr_Moving_Average'] = MSFT['Open'].rolling(window=20).mean()

#4. RSI
def calculate_rsi(data, window=20):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

MSFT['MSFT_RSI'] = calculate_rsi(MSFT['Close'])
MSFT['MSFT_Volume'] = MSFT['Volume']
MSFT = MSFT.between_time('09:30:00', '16:00:00')
missing_values_count = MSFT['MSFT_RSI'].isna().sum()
print(f"Number of missing values in 'ColumnName': {missing_values_count}")

MSFT.index = pd.to_datetime(MSFT.index)
nyse_calendar = mcal.get_calendar('NYSE')


start_date = pd.Timestamp('2022-11-21 09:30:00')
end_date = pd.Timestamp('2023-11-08 16:00:00')

schedule = nyse_calendar.schedule(start_date=start_date, end_date=end_date)
MSFT = MSFT[MSFT.index.normalize().isin(schedule.index.date)]

MSFT.to_csv('Filtered_Data/MSFT_filtered.csv')
MSFT.shape

Number of missing values in 'ColumnName': 19


(19197, 13)

In [191]:
####################Done MSFT####################

In [192]:
TSLA = pd.read_excel('Data/TSLA.xlsx', index_col=0)
TSLA.index = pd.to_datetime(TSLA.index)

#Feature Engineering
#1. Log Change
TSLA['TSLA_Log_Change'] = np.log(TSLA['Close'] / TSLA['Close'].shift(1))

#2. Bid-Ask Spread
TSLA['TSLA_High_Low_Spread'] = TSLA['High'] - TSLA['Low']
TSLA['TSLA_Volume'] = TSLA['Volume']

#3. Moving Average
TSLA['TSLA_1hr_Moving_Average'] = TSLA['Open'].rolling(window=20).mean()

#4. RSI
def calculate_rsi(data, window=20):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

TSLA['TSLA_RSI'] = calculate_rsi(TSLA['Close'])
TSLA['TSLA_Volume'] = TSLA['Volume']
TSLA = TSLA.between_time('09:30:00', '16:00:00')
missing_values_count = TSLA['TSLA_RSI'].isna().sum()
print(f"Number of missing values in 'ColumnName': {missing_values_count}")

TSLA.index = pd.to_datetime(TSLA.index)
nyse_calendar = mcal.get_calendar('NYSE')


start_date = pd.Timestamp('2022-11-21 09:30:00')
end_date = pd.Timestamp('2023-11-08 16:00:00')

schedule = nyse_calendar.schedule(start_date=start_date, end_date=end_date)
TSLA = TSLA[TSLA.index.normalize().isin(schedule.index.date)]

TSLA.to_csv('Filtered_Data/TSLA_filtered.csv')
TSLA.shape

Number of missing values in 'ColumnName': 19


(19197, 13)

In [193]:
####################Done TSLA####################

In [194]:
####################Done Stock Preprocessing####################

In [205]:
ARCA_TECH = pd.read_excel('Data/ARCA_TECH.xlsx', index_col=0)
ARCA_TECH.index = pd.to_datetime(ARCA_TECH.index)

#Feature Engineering
#1. Log Change
ARCA_TECH['ARCA_TECH_Log_Change'] = np.log(ARCA_TECH['Close'] / ARCA_TECH['Close'].shift(1))

#2. Moving Average
ARCA_TECH['ARCA_TECH_1hr_Moving_Average'] = ARCA_TECH['Close'].rolling(window=20).mean()

ARCA_TECH = ARCA_TECH.between_time('09:30:00', '16:00:00')
missing_values_count = ARCA_TECH['ARCA_TECH_1hr_Moving_Average'].isna().sum()
print(f"Number of missing values in 'ColumnName': {missing_values_count}")


ARCA_TECH.index = pd.to_datetime(ARCA_TECH.index)
nyse_calendar = mcal.get_calendar('NYSE')


start_date = pd.Timestamp('2022-11-21 09:30:00')
end_date = pd.Timestamp('2023-11-08 16:00:00')

schedule = nyse_calendar.schedule(start_date=start_date, end_date=end_date)
ARCA_TECH = ARCA_TECH[ARCA_TECH.index.normalize().isin(schedule.index.date)]


ARCA_TECH.to_csv('Filtered_Data/ARCA_TECH_filtered.csv')
ARCA_TECH.shape

Number of missing values in 'ColumnName': 19


(18884, 9)

In [196]:
####################Done Index Preprocessing####################

In [197]:
NASDAQ = pd.read_excel('Data/NASDAQ.xlsx', index_col=0)
NASDAQ.index = pd.to_datetime(NASDAQ.index)

#Feature Engineering
#1. Log Change
NASDAQ['NASDAQ_Log_Change'] = np.log(ARCA_TECH['Close'] / ARCA_TECH['Close'].shift(1))

#2. Moving Average
NASDAQ['NASDAQ_1hr_Moving_Average'] = NASDAQ['Close'].rolling(window=20).mean()

NASDAQ = NASDAQ.between_time('09:30:00', '16:00:00')
missing_values_count = NASDAQ['NASDAQ_1hr_Moving_Average'].isna().sum()
print(f"Number of missing values in 'ColumnName': {missing_values_count}")

NASDAQ.index = pd.to_datetime(NASDAQ.index)
nyse_calendar = mcal.get_calendar('NYSE')


start_date = pd.Timestamp('2022-11-21 09:30:00')
end_date = pd.Timestamp('2023-11-08 16:00:00')

schedule = nyse_calendar.schedule(start_date=start_date, end_date=end_date)
NASDAQ = NASDAQ[NASDAQ.index.normalize().isin(schedule.index.date)]


NASDAQ.to_csv('Filtered_Data/NASDAQ_filtered.csv')
NASDAQ.shape

Number of missing values in 'ColumnName': 19


(18836, 10)

In [198]:
OIL = pd.read_excel('Data/OIL Price.xlsx', index_col=0)
#Feature Engineering
#1. log return
OIL['OIL_Log_Change'] = np.log(OIL['Close'] / OIL['Close'].shift(1))
#2. Volume
OIL['OIL_Volume'] = OIL['Volume']
#3. Bid-Ask Spread
OIL['OIL_High_Low_Spread'] = OIL['Ask'] - OIL['Bid']

OIL = OIL.between_time('09:30:00', '16:00:00')

OIL.index = pd.to_datetime(OIL.index)
nyse_calendar = mcal.get_calendar('NYSE')


start_date = pd.Timestamp('2022-11-21 09:30:00')
end_date = pd.Timestamp('2023-11-08 16:00:00')

schedule = nyse_calendar.schedule(start_date=start_date, end_date=end_date)
OIL = OIL[OIL.index.normalize().isin(schedule.index.date)]


OIL.to_csv('Filtered_Data/OIL_filtered.csv')
OIL.shape




(18944, 13)

In [199]:
####################Done Commodities####################

In [203]:
# Convert index to datetime (if not already)
AAPL.index = pd.to_datetime(AAPL.index)
OIL.index = pd.to_datetime(OIL.index)

# Align the date ranges of both datasets
start_date = max(AAPL.index.min(), OIL.index.min())
end_date = min(AAPL.index.max(), OIL.index.max())
AAPL_aligned = AAPL.loc[start_date:end_date]
OIL_aligned = OIL.loc[start_date:end_date]

# Find timestamps in AAPL that are not in OIL
missing_in_AAPL = OIL_aligned.index.difference(AAPL_aligned.index)
print(missing_in_AAPL)

DatetimeIndex([], dtype='datetime64[ns]', name='Local Date', freq=None)
